In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated, Optional
from typing_extensions import TypedDict, Literal
from pymongo import MongoClient, errors
from dotenv import load_dotenv
import uuid
import json
import os

load_dotenv()

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
db = client['agentOrchestratorDB']

In [ ]:
def agent_creation():
    agents_collection = db['agents']

    agent_name = input("Enter Agent Name: ")
    agent_description = input("Enter Agent Description: ")
    agent_prompt = input("Enter Agent Prompt: ")
    
    agent_data = {
        "name": agent_name,
        "description": agent_description,
        "prompt": agent_prompt
    }

    result = agents_collection.insert_one(agent_data)

    print(f"Created agent with object id: {result.inserted_id}")

In [ ]:
def fetch_agents():
    agents_collection = db['agents']
    agents = agents_collection.find()
    available_agents = []
    for i, agent in enumerate(agents, start=1):
        available_agents.append({
            "reference_id": i,
            "agent_id": str(agent.get("_id")),
            "agent_name": agent.get("name"),
            "agent_description": agent.get("description"),
            "agent_prompt": agent.get("prompt")
        })
    
    return available_agents

In [ ]:
def display_agents(agents):
    print("Available Agents:\n")
    for agent in agents:
        print(f"  - Reference ID: {agent['reference_id']}, Agent ID: {agent['agent_id']}, Name: {agent['agent_name']}, Description: {agent['agent_description']}, Prompt: {agent["agent_prompt"][:30]}...{agent["agent_prompt"][-30:]}")

In [ ]:
def create_workflow():
    workdlows_collection = db['workflows']

    agents = fetch_agents()
    display_agents(agents)

    workflow_name = input("Enter Workflow Name: ")
    workflow_description = input("Enter Workflow Description: ")

    selected_ids_input = input("Enter comma-separated Agent Reference IDs to include in the workflow: ")
    try:
        selected_ids = [int(agent_id.strip()) for agent_id in selected_ids_input.split(',')]
    except ValueError:
        print("Invalid input. Please enter only numeric Agent Reference IDs.")
        return

    selected_agents = [agent for agent in agents if agent["reference_id"] in selected_ids]
    if len(selected_agents) != len(selected_ids):
        print("Some Agent IDs were not found. Please try again.")
        return

    workflow_nodes = []
    print("\nNow, define the connections between the selected agents.")
    for agent in selected_agents:
        while True:
            print(f"\nAgent {agent['reference_id']} - {agent['agent_name']}")
            conn_input = input("Enter connected Agent IDs (comma-separated) or press Enter for none: ")

            if conn_input.strip() == "":
                connects = []
                break
            try:
                connects = [int(cid.strip()) for cid in conn_input.split(',')]
            except ValueError:
                print("Invalid input. Please enter only numeric Agent Reference IDs.")
                continue

            if any(cid == agent["reference_id"] for cid in connects):
                print("❌ An agent cannot connect to itself. Please try again.")
                continue

            if all(cid in selected_ids for cid in connects):
                break
            else:
                print("❌ One or more Agent Reference IDs are invalid or not in the selected workflow. Please try again.")

        ref_to_agent = {agent["reference_id"]: agent["agent_id"] for agent in selected_agents}
        connects = [ref_to_agent[cid] for cid in connects if cid in ref_to_agent]

        workflow_nodes.append({
            "agent_id": agent["agent_id"],
            "name": agent["agent_name"],
            "description": agent["agent_description"],
            "connects": connects
        })

    print("\n✅ Workflow Created:")
    print(f"Name: {workflow_name}")
    print(f"Description: {workflow_description}")
    print("Nodes:")
    for node in workflow_nodes:
        print(f"  - Obj ID: {node['agent_id']}, Name: {node['name']}, Connects: {node['connects']}")

    workflow_data = {
        "workflow_name": workflow_name,
        "workflow_description": workflow_description,
        "workflow": workflow_nodes
    }

    result = workdlows_collection.insert_one(workflow_data)

    print(f"Created workflow with object id: {result.inserted_id}")

create_workflow()

In [ ]:
def fetch_workflows():
    workflows_collection = db['workflows']
    workflows = workflows_collection.find()

    available_workflows = []
    for i, workflow in enumerate(workflows, start=1):
        available_workflows.append({
            "reference_id": i,
            "workflow_id": str(workflow.get("_id")),
            "workflow_name": workflow.get("workflow_name"),
            "workflow_description": workflow.get("workflow_description"),
            "workflow": workflow.get("workflow")
        })
    
    return available_workflows

In [ ]:
def display_workflows(workflows):
    print("Available Workflows:\n")
    for workflow in workflows:
        print(f"  - Reference ID: {workflow['reference_id']}, Workflow ID: {workflow['workflow_id']}, Name: {workflow['workflow_name']}, Description: {workflow['workflow_description']}")

In [ ]:
def select_workflow():
    workflows = fetch_workflows()
    display_workflows(workflows)

    selected_id_input = input("Enter the Reference ID of a workflow to select: ")
    try:
        selected_id = int(selected_id_input)
    except ValueError:
        print("Invalid input. Please enter only a numeric Reference ID.")
        return

    selected_workflow = next((workflow for workflow in workflows if workflow["reference_id"] == selected_id), None)
    if selected_workflow is None:
        print("\n❌ No workflow found with that Reference ID. Please try again.")
        return

    return selected_workflow

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3
)

In [ ]:
def decide_next_node(state, selected_workflow):
    class DecidingSupervisorResponseFormat(TypedDict):
        next_node: Annotated[str, ..., "Node ID"]
        reasoning: Annotated[str, ..., "Reasoning"]
        instructions: Annotated[str, ..., "Instructions"]
        direct_response: Annotated[Optional[str], None, "Response to user"]
    
    deciding_supervisor_prompt = f"""
        You are a **supervising node** in a directed graph-based team workflow. Your job is to oversee and manage task delegation until the user's original task is fully completed.

        Each team member is a node in a graph, connected through the `connects` field. When a node finishes its part of the task, you—the supervisor—are informed. You must then decide which connected node should handle the next part of the task. Delegation is only allowed to nodes listed in the current node’s `connects`.

        ---

        ### 🎯 Primary Objective:

        Ensure the **user's task is fully and efficiently completed** by coordinating the workflow through the graph of capable agents.

        ---

        ### 🧠 Your Responsibilities:

        - Understand the **user’s overall task** and the **team structure**.
        - Break down the task into logical subtasks using each member’s `description`.
        - After each node completes a task:
        - Assess progress made so far.
        - Choose the next node from the available `connects`.
        - Provide the selected node with:
            - Relevant task context.
            - A clear description of what they need to do.
            - Any work already completed.

        - Avoid unnecessary or redundant assignments.
        - Make decisions based on:
        - **Skill alignment**
        - **Task continuity**
        - **Workflow logic**

        ---

        ### 🧾 You Will Receive:

        - The full **team structure**, including:
        - `name`, `node` (ID), `description`, and `connects` for each team member.
        - The **original user task**.
        - The **current state**, including:
        - The node that just completed its work.
        - Progress or outputs so far.
        - Chat history and context (always check this before deciding).

        ---

        ### 🧾 Your Output Must Include:

        1. **next_node**: ID of the next node (or `FINISH` if the task is done).
        2. **reasoning**: Justify why this node was chosen, based on their description and graph connections.
        3. **instructions**: Clear, contextual, and actionable guidance for the selected node.
        4. **direct_response** *(optional)*: A response to the user (e.g., greetings or status updates).

        ---

        ### ⚠️ Constraints:

        - You **can only choose from the current node’s `connects`**.
        - Never allow a node to delegate to itself.
        - If `connects` of previously used agent is empty, or the user just asked a general question (e.g., greetings or “what is your role”), respond with `FINISH` in `next_node`.
        - Once the workflow has started you will not finish until its complete (the previous agent has no connects).
        - Don’t assign tasks unless necessary—be efficient and purposeful.
        - Ensure each node receives enough information to pick up the task without confusion.

        ---

        ### 💬 Special Instruction for Role Questions:

        If and only if the user asks about your role or responsibilities (e.g., "What do you do?"), respond in the `direct_response` field by saying:

        > "I orchestrate the <given workflow> to ensure the user’s task is completed efficiently. Here's the current workflow I'm coordinating:"

        Then provide the workflow details from the `Given Workflow` section below.

        ---

        ### 🧩 Given Workflow:
        {json.dumps(selected_workflow, indent=2)}
    """

    messages = [SystemMessage(content=deciding_supervisor_prompt)] + state

    supervisor_response = llm.with_structured_output(DecidingSupervisorResponseFormat).invoke(messages)

    return supervisor_response

In [ ]:
def invoke_agent(agent, state):
    worker_agent = create_react_agent(
        model=llm,
        tools=[],
        prompt=ChatPromptTemplate([
            agent['agent_prompt'], 
            MessagesPlaceholder("messages")
        ]),
        name=agent['agent_name'],
    )
    agent_response = worker_agent.invoke({
        "messages": state
    })
    return agent_response["messages"][-1]

In [ ]:
def invoke_workflow():
    selected_workflow = select_workflow()
    agent_ids = [node["agent_id"] for node in selected_workflow["workflow"]] + ["FINISH"]

    user_prompt = input("You: ")
    state = [HumanMessage(content=user_prompt, name="user")]

    supervisor_response = ""
    while True:
        supervisor_response = decide_next_node(state, selected_workflow)

        if supervisor_response["next_node"] == "FINISH":
            print(supervisor_response["direct_response"])
            break
        elif supervisor_response["next_node"] in agent_ids:
            state.append(AIMessage(content=supervisor_response['instructions'], name="supervisor"))
            
            next_agent = next((agent for agent in fetch_agents() if agent["agent_id"] == supervisor_response["next_node"]), None)
            agent_response = invoke_agent(next_agent, state)
            agent_response.pretty_print()
            state.append(agent_response)
    
    return None

invoke_workflow()